In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import zipfile as zf
files = zf.ZipFile("/content/drive/MyDrive/BE Project/FBHM.zip", 'r')
files.extractall('memes')
files.close()

In [4]:
!pip install efficientnet_pytorch

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 112.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 85.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 60.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 83.8 MB/s eta 0:00:00
  Created wheel for efficientnet_pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16424 sha256=762f5a2537e0b72f3b42f3c89f5d606bc93e87407acf9a2c

In [6]:
from transformers import LxmertModel, LxmertTokenizer
from efficientnet_pytorch import EfficientNet
import torch.nn as nn

In [ ]:
import torch
import json
import os
import numpy as np
from PIL import Image
from transformers import LxmertTokenizer
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

In [ ]:
# ==========================
# CONFIGURATIONS
# ==========================
MODEL_PATH = "/content/drive/MyDrive/BE Project/best_model.pth"  # Path to trained model
TEST_JSON_PATH = "/content/Test_Data.json"  # Path to test dataset
IMG_DIR = "/content/memes/FBHM/data/"  # Directory containing images
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
MAX_LEN = 128
BATCH_SIZE = 32
TOKENIZER_NAME = "unc-nlp/lxmert-base-uncased"

In [ ]:
# ==========================
# LOAD MODEL
# ==========================
class ToxicMemeClassifier(torch.nn.Module):
    def __init__(self, num_classes=2):
        super().__init__()
        self.lxmert = LxmertModel.from_pretrained("unc-nlp/lxmert-base-uncased")

        # EfficientNet backbone for image processing
        self.cnn_backbone = EfficientNet.from_pretrained('efficientnet-b4')
        for param in self.cnn_backbone.parameters():
            param.requires_grad = False
        for block in self.cnn_backbone._blocks[-6:]:
            for param in block.parameters():
                param.requires_grad = True

        self.visual_fc = torch.nn.Linear(self.cnn_backbone._fc.in_features, 2048)
        self.cnn_backbone._fc = torch.nn.Identity()

        # Classifier head
        self.classifier = torch.nn.Sequential(
            torch.nn.Linear(self.lxmert.config.hidden_size, 512),
            torch.nn.GELU(),
            torch.nn.Dropout(0.3),
            torch.nn.Linear(512, num_classes)
        )

    def forward(self, input_ids, attention_mask, images):
        images = images.float()
        visual_feats = self.cnn_backbone(images)
        visual_feats = self.visual_fc(visual_feats).unsqueeze(1)

        batch_size = visual_feats.size(0)
        visual_pos = torch.tensor([[0, 0, 1, 1]], device=DEVICE).repeat(batch_size, 1, 1).float()

        outputs = self.lxmert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            visual_feats=visual_feats,
            visual_pos=visual_pos,
        )
        return self.classifier(outputs.pooled_output)

# Load trained model
model = ToxicMemeClassifier()
model.load_state_dict(torch.load(MODEL_PATH, map_location=DEVICE))
model.to(DEVICE)
model.eval()

In [ ]:
# ==========================
# PREPROCESSING FUNCTIONS
# ==========================
# Load tokenizer
tokenizer = LxmertTokenizer.from_pretrained(TOKENIZER_NAME)

# Image transformations (must match training pipeline)
image_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])

def preprocess_text(text):
    encoding = tokenizer(text, max_length=MAX_LEN, padding="max_length", truncation=True, return_tensors="pt")
    return encoding["input_ids"].squeeze(0), encoding["attention_mask"].squeeze(0)

def preprocess_image(img_path):
    image = Image.open(img_path).convert("RGB")
    return image_transform(image)

In [ ]:
# ==========================
# TEST DATASET CLASS
# ==========================
class TestDataset(Dataset):
    def __init__(self, json_path, img_dir, tokenizer, max_len, transform):
        with open(json_path, "r") as f:
            self.data = json.load(f)

        self.img_dir = img_dir
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        text = item["text"]
        img_path = os.path.join(self.img_dir, item["img"].lstrip("/"))

        if not os.path.exists(img_path):
            raise FileNotFoundError(f"Image not found: {img_path}")

        input_ids, attention_mask = preprocess_text(text)
        image = preprocess_image(img_path)

        return {
            "input_ids": input_ids.long(),
            "attention_mask": attention_mask.long(),
            "image": image,
            "label": torch.tensor(item.get("label", 0), dtype=torch.long),  # Assuming label exists in test.json
        }

In [ ]:
# ==========================
# LOAD TEST DATA
# ==========================
test_dataset = TestDataset(TEST_JSON_PATH, IMG_DIR, tokenizer, MAX_LEN, image_transform)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
# ==========================
# EVALUATE MODEL
# ==========================
y_true = []
y_pred = []
criterion = torch.nn.CrossEntropyLoss()

total_loss = 0
with torch.no_grad():
    for batch in test_loader:
        input_ids = batch["input_ids"].to(DEVICE)
        attention_mask = batch["attention_mask"].to(DEVICE)
        images = batch["image"].to(DEVICE)
        labels = batch["label"].to(DEVICE)

        outputs = model(input_ids, attention_mask, images)
        loss = criterion(outputs, labels)
        total_loss += loss.item()

        preds = torch.argmax(outputs, dim=1).cpu().numpy()
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(preds)

In [ ]:
# ==========================
# COMPUTE METRICS
# ==========================
accuracy = accuracy_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred, average="weighted")
precision = precision_score(y_true, y_pred, average="weighted", zero_division=0)
recall = recall_score(y_true, y_pred, average="weighted", zero_division=0)

In [7]:
# ==========================
# PRINT RESULTS
# ==========================
print("\nTest Evaluation Metrics:")
print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.01G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.01G [00:00<?, ?B/s]

Some weights of the model checkpoint at unc-nlp/lxmert-base-uncased were not used when initializing LxmertModel: ['answer_head.logit_fc.0.bias', 'answer_head.logit_fc.0.weight', 'answer_head.logit_fc.2.bias', 'answer_head.logit_fc.2.weight', 'answer_head.logit_fc.3.bias', 'answer_head.logit_fc.3.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'obj_predict_head.decoder_dict.attr.bias', 'obj_predict_head.decoder_dict.attr.weight', 'obj_predict_head.decoder_dict.feat.bias', 'obj_predict_head.decoder_dict.feat.weight', 'obj_predict_head.decoder_dict.obj.bias', 'obj_predict_head.decoder_dict.obj.weight', 'obj_predict_head.transform.LayerNorm.bias', 'obj_predict_head.transform.LayerNorm.weight', 'obj_predict_head.transform.dense.bias', 'obj_pred

...

tokenizer_config.json:   0%|          | 0.00/153 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]


Test Evaluation Metrics:
Accuracy: 0.5500
F1 Score: 0.7097
Precision: 1.0000
Recall: 0.5500
